In [1]:
# OpenAI API Key 정의
# **주의!** OpenAI API Key 는 외부에 노출하지 않도록 주의한다. 
# 4월 17일 이후에는 이전에 사용하던 사용자 키(User Key)보다 프로젝트별로 구분할 수 있는 프로젝트 키(Project Key)를 활용하는 것을 권장
# 일반적으로 시스템의 환경변수에 숨겨놓고 소스코드에는 노출하지 않지만, 
# 해당 과정은 파이썬 활용에 관한 내용이므로 예제에서는 생략한다.
# [모델 문서](https://platform.openai.com/docs/models)에는 사용가능한 모델의 이름이 나열되어있고, OpenAI에서 모델을 업데이트할 때마다 이곳에 명시하므로 자주 확인하면 좋다.

OPENAI_API_KEY = "YOUR_OPENAI_PROJECT_KEY"
OPENAI_MODEL = "gpt-4-turbo"

In [2]:
# OpenAI API 패키지 import
try:
    import openai

except ModuleNotFoundError:
    # OpenAI API 패키지 설치
    !pip install -r requirements.txt

finally:
    import openai

if "1.23.1" != openai.VERSION:
    raise "예제는 2024-04-17 update 버전을 기준으로 작성하였습니다."

## 어시스턴트 애플리케이션 구성 요소 (기본)

1. 스레드 생성
2. 어시스턴트 생성
3. 사용자 메시지 생성
4. 런 생성 및 결과 대기
5. 런 상태 확인 및 스레드 메시지 목록 확인
6. 어시스턴트 삭제
7. 스레드 삭제

In [3]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# 1. 스레드 생성
thread = client.beta.threads.create()
print(f"스레드 생성: {thread.id}")

# 2. 어시스턴트 생성
assistant = client.beta.assistants.create(
    name="수학 도우미",
    instructions="여러분은 개인 수학 도우미입니다. 수학 문제에 답하는 코드를 작성하고 실행하세요.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo",
)
print(f"어시스턴트 생성: {assistant.id}")

# 3. 사용자 메시지 생성
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="'3x + 11 = 14'라는 방정식을 풀어야 합니다. 도와주실 수 있나요?"
)
print(f"사용자 메시지 생성: {message.id}")

# 4. 런 생성 및 결과 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# 5. 런 상태 확인 및 스레드 메시지 목록 확인
if run.status == 'completed': 
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    
    messages.data.reverse()
    for message in messages.data:
        print(f"{message.role}> ")
        for content in message.content:
            print(content.text.value)
else:
    print(run.status)
    
# 6. 어시스턴트 삭제
res = client.beta.assistants.delete(
    assistant_id=assistant.id
)
print(f"assistant deleted: {res.deleted}")

# 7. 스레드 삭제
res = client.beta.threads.delete(
    thread_id=thread.id
)
print(f"thread deleted: {res.deleted}")

스레드 생성: thread_R2W0lM3rpJANGXbZlaXvGzPs
어시스턴트 생성: asst_xU3ZYRnHUXQC9MCoXIjgtvtG
사용자 메시지 생성: msg_dd6IzRYCNCaHHbUNQzrBM6pw
user> 
'3x + 11 = 14'라는 방정식을 풀어야 합니다. 도와주실 수 있나요?
assistant> 
물론입니다! 우리는 방정식 \(3x + 11 = 14\)을 풀어서 \(x\)에 대한 값을 찾아야 합니다.

첫 번째 단계로, \(3x\)항을 분리하기 위해 양변에서 11을 빼줍니다. 그런 다음, 결과로 얻어진 식을 3으로 나누어 \(x\)의 값을 구합니다. 계산을 진행해 보겠습니다.
assistant> 
방정식 \(3x + 11 = 14\)을 풀면 \(x\)의 값은 1입니다. 따라서 \(x = 1\)이 정답입니다. 다른 질문이 있으시면 언제든지 알려주세요!
assistant deleted: True
thread deleted: True


## 어시스턴트 애플리케이션 구성 요소 (스트림 활용)

1. 스레드 생성
2. 어시스턴트 생성
3. 사용자 메시지 생성
4. 런 스트림 생성
5. 런 스트림 이벤트에 따른 동작 관찰
6. 어시스턴트 삭제
7. 스레드 삭제

In [4]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# 1. 스레드 생성
thread = client.beta.threads.create()
print(f"스레드 생성: {thread.id}")

# 2. 어시스턴트 생성
assistant = client.beta.assistants.create(
    name="수학 도우미",
    instructions="여러분은 개인 수학 도우미입니다. 수학 문제에 답하는 코드를 작성하고 실행하세요.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo",
)
print(f"어시스턴트 생성: {assistant.id}")

# 3. 사용자 메시지 생성
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="'3x + 11 = 14'라는 방정식을 풀어야 합니다. 도와주실 수 있나요?"
)
print(f"사용자 메시지 생성: {message.id}")

# 4. 런 스트림 생성
print(f"런 생성")
run_stream = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    stream=True
)

# 5. 런 스트림의 이벤트에 따라 런 스탭 및 메시지 조각을 확인
for event in run_stream:
    data = event.data
    eventobj = data.object
      
    if "thread.run" == eventobj:
        print(f"{event.event}> status: {data.status}")

    elif "thread.run.step" == eventobj:
        if "message_creation" == data.type:
            print(f"{event.event}> status: {data.status}, type: {data.type}")

        elif "tool_calls" == data.type:
            if "thread.run.step.created" == event.event:
                print(f"{event.event}> status: {data.status}, type: {data.type}")

            elif "thread.run.step.in_progress" == event.event:
                print(f"어시스턴트 도구 시작>", flush=True)
                
            elif "thread.run.step.completed" == event.event:
                print(f"어시스턴트 도구 종료>", flush=True)
                print(f"{event.event}> status: {data.status}, type: {data.type}")
        
    elif "thread.run.step.delta" == eventobj:
        delta = data.delta
        if "tool_calls" == delta.step_details.type:
            tool_calls = delta.step_details.tool_calls
            for content in tool_calls:
                if "code_interpreter" == content.type:
                    if content.code_interpreter.input:
                        print(content.code_interpreter.input, end="", flush=True)

                    if content.code_interpreter.outputs:
                        for output in content.code_interpreter.outputs:
                            if "logs" == output.type:
                                print(f"\n{output.logs}", flush=True)
        
    elif "thread.message" == eventobj:
        if "thread.message.created" == event.event:
            print(f"{event.event}> status: {data.status}, role: {data.role}")
            
        if "thread.message.in_progress" == event.event:
            print(f"{data.role} 응답 시작>", flush=True)

        elif "thread.message.completed" == event.event:
            print(f"\n{data.role} 응답 종료>", flush=True)
            print(f"{event.event}> status: {data.status}, role: {data.role}")
        
    elif "thread.message.delta" == eventobj:
        delta = data.delta
        for content in delta.content:
            print(f"{content.text.value}", end="", flush=True)
        
    else:
        print(data.to_dict())

# 6. 어시스턴트 삭제
res = client.beta.assistants.delete(
    assistant_id=assistant.id
)
print(f"assistant deleted: {res.deleted}")

# 7. 스레드 삭제
res = client.beta.threads.delete(
    thread_id=thread.id
)
print(f"thread deleted: {res.deleted}")

스레드 생성: thread_5VZj6BEKfWDQmFnuemu8gqSn
어시스턴트 생성: asst_sW5v8OzQr07NYCoUqXG5LUAK
사용자 메시지 생성: msg_wqn5xoPVZnSgboNOt3S3RZXa
런 생성
thread.run.created> status: queued
thread.run.queued> status: queued
thread.run.in_progress> status: in_progress
thread.run.step.created> status: in_progress, type: message_creation
thread.run.step.in_progress> status: in_progress, type: message_creation
thread.message.created> status: in_progress, role: assistant
assistant 응답 시작>
물론입니다. 이 방정식에서 변수 \(x\)의 값을 찾아야 합니다. 방정식을 풀기 위해서는 다음 단계를 따르겠습니다:

1. 방정식의 양변에서 11을 뺍니다.
2. 나머지를 3으로 나눕니다.

이제 이 단계들을 계산을 통해 실행하겠습니다.
assistant 응답 종료>
thread.message.completed> status: completed, role: assistant
thread.run.step.completed> status: completed, type: message_creation
thread.run.step.created> status: in_progress, type: tool_calls
어시스턴트 도구 시작>
from sympy import symbols, Eq, solve

x = symbols('x')
equation = Eq(3 * x + 11, 14)

# Solve the equation
solution = solve(equation, x)
solution
[1]
어시스턴트 도구 종료>
thread.run.step.comple